In [39]:
from gmm_denoiser import GMMDenoiser
from regularized_denoisers import denoise_l2
from skimage.restoration import denoise_wavelet
from utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [40]:
df = pd.read_excel("data/PD data.xlsx")

In [41]:
train_signals = [
    np.array(df["s1sensor1"]),
    np.array(df["s1sensor2"]),
    np.array(df["s1sensor3"]),
    np.array(df["s1sensor4"]),
    np.array(df["s1sensor5"]),
]

In [42]:
test_signals = [
    np.array(df["s2sensor1"]),
    np.array(df["s2sensor2"]),
    np.array(df["s2sensor3"]),
    np.array(df["s2sensor4"]),
    np.array(df["s2sensor5"]),
]

In [43]:
test_snrs = [-10, -5, 0, 5, 10]
denoisers = ["none", "wavelet", "dae", "gmm"]
results = np.zeros((len(test_signals), len(test_snrs), len(denoisers), len(test_signals[0])))

In [53]:
import keras
model_old = keras.Sequential([
    keras.layers.Input(shape=(2500, 1)),
    keras.layers.Conv1D(filters=32, kernel_size=9, activation="relu", padding="same"),
    keras.layers.MaxPooling1D(2),
    keras.layers.Conv1D(filters=32, kernel_size=9, activation="relu", padding="same"),
    keras.layers.MaxPooling1D(2),

    keras.layers.Conv1DTranspose(filters=32, kernel_size=9, strides=2, activation="relu", padding="same"),
    keras.layers.Conv1DTranspose(filters=32, kernel_size=9, strides=2, activation="relu", padding="same"),
    keras.layers.Conv1D(1, 9, padding="same")
])
model_old.compile(optimizer="adam", loss="mse")

model = keras.Sequential([
    keras.layers.Input(shape=(2500, 1)),
    keras.layers.Conv1D(filters=32, kernel_size=150, activation="relu", padding="same"),
    keras.layers.MaxPooling1D(2),
    keras.layers.Conv1D(filters=32, kernel_size=10, activation="relu", padding="same"),
    keras.layers.MaxPooling1D(2),

    keras.layers.Conv1DTranspose(filters=32, kernel_size=10, strides=2, activation="relu", padding="same"),
    keras.layers.Conv1DTranspose(filters=32, kernel_size=150, strides=2, activation="relu", padding="same"),
    keras.layers.Conv1D(1, 9, padding="same")
])
model.compile(optimizer="adam", loss="mse")

model_X = []; model_y = []
for ts in train_signals:
    model_X.append(ts)
    model_y.append(ts)
    for snr in [-10, -5, 0, 5, 10]:
      model_X.append(awgn(ts, snr))
      model_y.append(ts)
model_X = np.array(model_X).reshape(30, 2500, 1)
model_y = np.array(model_y).reshape(30, 2500, 1)

model_old.fit(model_X, model_y, batch_size=1, epochs=10)
# model.summary()
model.fit(model_X, model_y, batch_size=1, epochs=10)


Epoch 1/10


30/30 [==============================] - 1s 6ms/step - loss: 0.0572
Epoch 2/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0256
Epoch 3/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0165
Epoch 4/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0108
Epoch 5/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0119
Epoch 6/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0109
Epoch 7/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0098
Epoch 8/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0103
Epoch 9/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0132
Epoch 10/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0102
Epoch 1/10
30/30 [==============================] - 2s 30ms/step - loss: 0.0432
Epoch 2/10
30/30 [==============================] - 1s 29ms/step - loss: 0.0112
Epoch 3/10
30/30 [==============================] - 1s 30ms/

In [45]:
denoiser = GMMDenoiser(gmm_n_components=10, patch_size=150, train_signals=train_signals)
denoiser.fit()

In [54]:
for i, nf in enumerate(test_signals):
    for j, snr in enumerate(test_snrs):
        print(i, j)
        ns = awgn(nf, snr)
        results[i, j, 0, :] = ns
        results[i, j, 1, :] = denoise_wavelet(ns, method="BayesShrink", mode="soft", wavelet_levels=3, wavelet="sym8", rescale_sigma="True")
        # results[i, j, 2, :] = np.array(model_old(ns.reshape(2500, 1).reshape(1, -1))).reshape((2500,))
        results[i, j, 2, :] = np.array(model(ns.reshape(2500, 1).reshape(1, -1))).reshape((2500,))
        results[i, j, 3, :] = denoiser.denoise(ns, snr_db_est=snr)


0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4


In [55]:
print(f"{'method':^15}|{'snr':^8}|{'psnr':^8}|{'mse':^8}|{'ssim':^8}|{'tv':^8}")
csv_s = f"{'method':^15}|{'snr':^8}|{'psnr':^8}|{'mse':^8}|{'ssim':^8}|{'tv':^8}".replace("|",",").replace(" ", "")
metrics = ["snr", "psnr", "mse", "ssim", "tv"]
metric_arr = np.zeros((len(test_signals), len(test_snrs), len(denoisers), len(metrics)))
for i, nf in enumerate(test_signals):
    for j, snr in enumerate(test_snrs):
        ns = results[i, j, 0]
        for k, (name, dns) in enumerate(zip(denoisers, results[i, j])):
            metric_arr[i, j, k, 0] = calc_snr(nf, dns)
            metric_arr[i, j, k, 1] = psnr(nf, dns)
            metric_arr[i, j, k, 2] = mse(nf, dns)
            metric_arr[i, j, k, 3] = ssim(nf, dns)
            metric_arr[i, j, k, 4] = tv(dns)
            print(f"{name:<15}|{calc_snr(nf, dns):>8.2f}|{psnr(nf, dns):>8.2f}|{mse(nf, dns):>8.4f}|{ssim(nf, dns):>8.4f}|{tv(dns):>8.1f}")
            csv_s += f"\n{name:<15}|{calc_snr(nf, dns):>8.2f}|{psnr(nf, dns):>8.2f}|{mse(nf, dns):>8.4f}|{ssim(nf, dns):>8.4f}|{tv(dns):>8.1f}".replace("|",",").replace(" ", "")
        print("-"*60)


    method     |  snr   |  psnr  |  mse   |  ssim  |   tv   
none           |   -9.88|    2.38|  3.3362|  0.0047|  5167.6
wavelet        |   -0.68|   11.58|  0.4012|  0.1501|   292.8
dae            |    8.04|   20.30|  0.0539|  0.5226|    77.0
gmm            |    9.71|   21.97|  0.0367|  0.6978|    46.5
------------------------------------------------------------
none           |   -5.12|    7.14|  1.1148|  0.0153|  2981.7
wavelet        |    4.25|   16.51|  0.1289|  0.3108|   197.9
dae            |   13.49|   25.75|  0.0154|  0.8547|    63.3
gmm            |   14.20|   26.46|  0.0131|  0.8365|    48.0
------------------------------------------------------------
none           |   -0.05|   12.21|  0.3474|  0.0530|  1673.8
wavelet        |    8.64|   20.90|  0.0470|  0.5360|   109.0
dae            |   14.86|   27.12|  0.0112|  0.8864|    55.0
gmm            |   14.83|   27.09|  0.0113|  0.8421|    50.9
------------------------------------------------------------
none           |    5.12

In [56]:
plt.ioff()
import pathlib
pathlib.Path("out/pd_out/persigsnr").mkdir(exist_ok=True, parents=True)
for i, nf in enumerate(test_signals):
    for j, snr in enumerate(test_snrs):
        print(i, j)
        fig, axs = plt.subplots(1, 4, figsize=(12, 3))
        axs[0].set_title("Noisy")
        axs[1].set_title("Wavelet Denoised")
        axs[2].set_title("DAE Denoised")
        axs[3].set_title("GMM Denoised")
        for k, dns in enumerate(results[i, j]):
            axs[k].plot(dns, linewidth=0.5, label=("noisy" if k == 0 else "denoised"))
            axs[k].plot(nf, linewidth=0.5, label="clean")
            axs[k].text(0.5, 0.8, f"snr = {calc_snr(nf, dns):.2f}", transform=axs[k].transAxes)
            axs[k].legend(loc="lower right")
        fig.savefig(f"out/pd_out/persigsnr/sig{i}snr{snr}.png")
        plt.close(fig)
plt.ion()


0 0


0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4


In [59]:
with open("out/indices.csv", "w") as f:
    f.write(csv_s)

In [57]:
print(f"average increase in snr:")
for j, snr in enumerate(test_snrs):
    m_snrs = metric_arr[:, j, :, 0]
    none, wav, dae, gmm = (m_snrs - m_snrs[:, 0].reshape(-1, 1)).mean(axis=0)
    
    print(f"original: {snr:>3}; wavelet: {wav:>5.2f}; dae: {dae:>5.2f}; gmm: {gmm:>5.2f}")
    

average increase in snr:
original: -10; wavelet:  9.11; dae: 18.61; gmm: 18.59
original:  -5; wavelet:  9.01; dae: 16.68; gmm: 17.78
original:   0; wavelet:  9.01; dae: 14.67; gmm: 16.35
original:   5; wavelet:  9.12; dae: 10.78; gmm: 16.34
original:  10; wavelet:  9.09; dae:  6.09; gmm: 15.49


In [58]:
model.save("model.keras")